# L-shaped algorithm example

We illustrate in this notebook the example 2 in Chapter 5, page 188, from Birge and Louveaux (2011), "Introduction to Stochastic Programming", 2nd edition, Springer.

In [3]:
using StochasticPrograms
using HiGHS

First, we construct the 2-stage model.

In [4]:
@stochastic_model simple_model begin
    @stage 1 begin
        @decision(simple_model, x >= 0)
        @objective(simple_model, Min, 0*x)
    end
    @stage 2 begin
        @known(simple_model, x)
        @uncertain ξ
        @recourse(simple_model, 0 <= y[i in 1:2])
        @objective(simple_model, Min, y[1] + y[2])
        @constraint(simple_model, y[1] - y[2] == ξ - x)
    end
end

Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)


We now define three scenarios of equal probability.

In [5]:
ξ1 = @scenario ξ = 1 probability = 1/3
ξ2 = @scenario ξ = 2 probability = 1/3
ξ3 = @scenario ξ = 4 probability = 1/3

ξ = [ξ1, ξ2, ξ3]

3-element Vector{Scenario{NamedTuple{(:ξ,), Tuple{Int64}}}}:
 Scenario with probability 0.3333333333333333
  ξ: 1
 Scenario with probability 0.3333333333333333
  ξ: 2
 Scenario with probability 0.3333333333333333
  ξ: 4

## Deterministic equivalent

In order to have a solvable program, we must instantiate the model. By default, `StochasticPrograms.jl` will generate the derministic equivalent form.

In [6]:
sp = instantiate(simple_model, ξ, optimizer = HiGHS.Optimizer)

Stochastic program with:
 * 1 decision variable
 * 2 recourse variables
 * 3 scenarios of type Scenario
Structure: Deterministic equivalent
Solver name: HiGHS

We can check the model by printing it on the screen.

In [7]:
println(sp)

Deterministic equivalent problem
Min 0.3333333333333333 y₁[1] + 0.3333333333333333 y₁[2] + 0.3333333333333333 y₂[1] + 0.3333333333333333 y₂[2] + 0.3333333333333333 y₃[1] + 0.3333333333333333 y₃[2]
Subject to
 x in Decisions
 y₁[1] in RecourseDecisions
 y₁[2] in RecourseDecisions
 y₂[1] in RecourseDecisions
 y₂[2] in RecourseDecisions
 y₃[1] in RecourseDecisions
 y₃[2] in RecourseDecisions
 x >= 0.0
 y₁[1] >= 0.0
 y₁[2] >= 0.0
 y₂[1] >= 0.0
 y₂[2] >= 0.0
 y₃[1] >= 0.0
 y₃[2] >= 0.0
 x + y₁[1] - y₁[2] == 1.0
 x + y₂[1] - y₂[2] == 2.0
 x + y₃[1] - y₃[2] == 4.0
Solver name: HiGHS


We now solve it.

In [8]:
optimize!(sp)

Presolving model
3 rows, 7 cols, 9 nonzeros
3 rows, 7 cols, 9 nonzeros
Presolve : Reductions: rows 3(-0); columns 7(-0); elements 9(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(7) 0s
          3     1.0000000000e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 3
Objective value     :  1.0000000000e+00
HiGHS run time      :          0.00


We can check the first stage solution with the method `value()`.

In [9]:
value(sp[1,:x])

2.0

## L-shaped

We now express the program as a 2-stage optimization problem, and explore various variants of the L-shaped decomposition algorithm.

### Multi cut

In [10]:
sp_lshaped = instantiate(simple_model, ξ, optimizer = LShaped.Optimizer)

Stochastic program with:
 * 1 decision variable
 * 2 recourse variables
 * 3 scenarios of type Scenario
Structure: Stage-decomposition
Solver name: L-shaped with disaggregate cuts

In [11]:
set_optimizer_attribute(sp_lshaped, MasterOptimizer(), HiGHS.Optimizer)
set_optimizer_attribute(sp_lshaped, SubProblemOptimizer(), HiGHS.Optimizer)

In [12]:
optimize!(sp_lshaped)

Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-2); columns 0(-3); elements 0(-4) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  2.6998783506e-01
HiGHS run time      :          0.00
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-2); columns 0(-3); elements 0(-4) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  1.2699878351e+00
HiGHS run time      :          0.00
Presolving model
0 rows, 0 cols, 0 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-2); columns 0(-3); elements 0(-4) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  3.2699878351e+00
HiGHS run time      :          0.00
Presolving model
Solving the original LP wi

L-Shaped Gap  Time: 0:00:03 (1 iterations)
  Objective:          -Inf
  Early termination:  DUAL_INFEASIBLE
  Number of cuts:     3
  Iterations:         1


The master problem is unbounded from below as $K_1$ is not bounded.

Let modify the first-stage feasible set by imposing a large bound on $x$, here $10^9$.

In [13]:
@stochastic_model simple_model begin
    @stage 1 begin
        @decision(simple_model, 0 <= x <= 1e9)
        @objective(simple_model, Min, 0*x)
    end
    @stage 2 begin
        @known(simple_model, x)
        @uncertain ξ
        @recourse(simple_model, 0 <= y[i in 1:2])
        @objective(simple_model, Min, y[1] + y[2])
        @constraint(simple_model, y[1] - y[2] == ξ - x)
    end
end

Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)


In [14]:
sp_lshaped = instantiate(simple_model, ξ, optimizer = LShaped.Optimizer)
set_optimizer_attribute(sp_lshaped, MasterOptimizer(), HiGHS.Optimizer)
set_optimizer_attribute(sp_lshaped, SubProblemOptimizer(), HiGHS.Optimizer)

println(sp_lshaped)

First-stage 
Min 0
Subject to
 x in Decisions
 x >= 0.0
 x <= 1.0e9

Second-stage 
Subproblem 1 (p = 0.33):
Min y[1] + y[2]
Subject to
 x in Known(value = 0.0)
 y[1] in RecourseDecisions
 y[2] in RecourseDecisions
 y[1] >= 0.0
 y[2] >= 0.0
 x + y[1] - y[2] == 1.0

Subproblem 2 (p = 0.33):
Min y[1] + y[2]
Subject to
 x in Known(value = 0.0)
 y[1] in RecourseDecisions
 y[2] in RecourseDecisions
 y[1] >= 0.0
 y[2] >= 0.0
 x + y[1] - y[2] == 2.0

Subproblem 3 (p = 0.33):
Min y[1] + y[2]
Subject to
 x in Known(value = 0.0)
 y[1] in RecourseDecisions
 y[2] in RecourseDecisions
 y[1] >= 0.0
 y[2] >= 0.0
 x + y[1] - y[2] == 4.0

Solver name: L-shaped with disaggregate cuts


We now solve the program.

In [15]:
optimize!(sp_lshaped)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\slash\.julia\packages\ProgressMeter\sN2xr\src\ProgressMeter.jl:618
L-Shaped Gap  Time: 0:00:00 (3 iterations)
  Objective:       1.0000000099341073
  Gap:             0.0
  Number of cuts:  7
  Iterations:      3


The first-stage solution is

In [16]:
value(sp_lshaped[1,:x])

2.0000000298023224

### Single Cut

Let's check what happens when the single cut variant is used.

In [17]:
set_optimizer_attribute(sp_lshaped, Aggregator(), Aggregate())
optimize!(sp_lshaped)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter C:\Users\slash\.julia\packages\ProgressMeter\sN2xr\src\ProgressMeter.jl:618
L-Shaped Gap  Time: 0:00:00 (5 iterations)
  Objective:       1.0
  Gap:             0.0
  Number of cuts:  4
  Iterations:      5


More iterations are needed, to obtain a similar first-stage solution.

In [18]:
value(sp_lshaped[1,:x])

1.9999999999999996